In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

In [4]:
df = pd.read_csv('personality_datasert.csv')
df.columns

print(f"Dataset shape: {df.shape}")
print(df.head())
print(f"Data Information:{df.info()}")

Dataset shape: (2900, 8)
   Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
0               4.0         No                      4.0            6.0   
1               9.0        Yes                      0.0            0.0   
2               9.0        Yes                      1.0            2.0   
3               0.0         No                      6.0            7.0   
4               3.0         No                      9.0            4.0   

  Drained_after_socializing  Friends_circle_size  Post_frequency Personality  
0                        No                 13.0             5.0   Extrovert  
1                       Yes                  0.0             3.0   Introvert  
2                       Yes                  5.0             2.0   Introvert  
3                        No                 14.0             8.0   Extrovert  
4                        No                  8.0             5.0   Extrovert  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 e

In [5]:
numeric_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 
                'Friends_circle_size', 'Post_frequency']

binary_cols = ['Stage_fear', 'Drained_after_socializing']

In [6]:
print(f"Missing values:{df.isnull().sum()}")
print()
print(f"Duplicate values:{df.duplicated().sum()}")
print()
print(f"Class distribution:{df['Personality'].value_counts()}")

Missing values:Time_spent_Alone             0
Stage_fear                   0
Social_event_attendance      0
Going_outside                0
Drained_after_socializing    0
Friends_circle_size          0
Post_frequency               0
Personality                  0
dtype: int64

Duplicate values:402

Class distribution:Personality
Extrovert    1491
Introvert    1409
Name: count, dtype: int64
